In [3]:
from hstrader import HsTrader
from hstrader.models import Resolution
import os
from dotenv import load_dotenv
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets, VBox
from IPython.display import display
#from datetime import datetime


In [5]:
# Load environment variables from a .env file
load_dotenv()

# Get the CLIENT_ID from the environment variables
id = os.getenv('CLIENT_ID')

# Get the CLIENT_SECRET from the environment variables
secret = os.getenv('CLIENT_SECRET')

# Initialize the HsTrader client with the client ID and secret
client = HsTrader(id, secret)

In [6]:
def get_fibonacci_levels(high, low, uptrend):
    """
    Calculate Fibonacci retracement levels.
    
    Parameters
    ---------
    high (float): The highest price in the range.
    low (float): The lowest price in the range.
    uptrend (bool): True for uptrend, False for downtrend.
    
    Returns:
    dict: A dictionary with Fibonacci retracement levels.
    """
    difference = high - low
    if uptrend:
        return {
            '0.00%': high,
            '23.6%': high - difference * 0.236,
            '38.2%': high - difference * 0.382,
            '50.0%': high - difference * 0.5,
            '61.8%': high - difference * 0.618,
            '100.0%': low
        }
    else:
        return {
            '100.0%': high,
            '61.8%': low + difference * 0.618,
            '50.0%': low + difference * 0.5,
            '38.2%': low + difference * 0.382,
            '23.6%': low + difference * 0.236,
            '0.00%': low
        }

In [7]:
def plot_fibonacci_retracement(df, fibonacci_df, levels, ticker):
    """
    Plots Fibonacci retracement levels on a candlestick chart.

    Parameters
    -----------
        df (DataFrame): DataFrame containing OHLC price data.
        fibonacci_df (DataFrame): A slice of the original DataFrame with calculated Fibonacci levels.
        levels (dict): Dictionary of Fibonacci retracement levels.
        ticker (str): Ticker symbol for the plot title.
    """
    fig = go.Figure()

    # Add candlestick chart
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='Candlestick'
    ))

    # Add Fibonacci levels and filled sections
    colors = ['rgba(128, 0, 128, 0.2)', 'rgba(0, 0, 255, 0.2)', 'rgba(0, 128, 0, 0.2)', 'rgba(255, 0, 0, 0.2)', 'rgba(255, 165, 0, 0.2)']
    level_values = list(levels.values())
    labels = list(levels.keys())

    for i in range(len(level_values)):
        # Draw the level lines
        fig.add_trace(go.Scatter(
            x=[fibonacci_df.index.min(), fibonacci_df.index.max()],
            y=[level_values[i], level_values[i]],
            mode='lines',
            line=dict(dash='dash', width=0.7, color=colors[i % len(colors)]),
            name=labels[i],
            showlegend=True,
            legendgroup=labels[i],
            hoverinfo='skip'
        ))

        # Add level labels
        fig.add_trace(go.Scatter(
            x=[fibonacci_df.index.max()],
            y=[level_values[i]],
            text=[labels[i]],
            mode='text',
            showlegend=False,
            legendgroup=labels[i],
            hoverinfo='skip'
        ))

        # Fill the area between levels
        if i > 0:
            fig.add_trace(go.Scatter(
                x=fibonacci_df.index.tolist() + fibonacci_df.index[::-1].tolist(),
                y=[level_values[i-1]] * len(fibonacci_df.index) + [level_values[i]] * len(fibonacci_df.index),
                fill='toself',
                fillcolor=colors[i % len(colors)],
                line=dict(width=0),
                name=f'{labels[i]} Fill',
                showlegend=True,
                hoverinfo='skip'
            ))

    fig.update_layout(
        title=f'Fibonacci Retracement for {ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        template='plotly_white',
        legend=dict(
            title='Fibonacci Levels',
            orientation='h',
            yanchor='bottom',
            y=1.1,
            xanchor='right',
            x=1
        ),
        margin=dict(t=150)
    )

    fig.show()


In [8]:
def fib_retrace(ticker):
    """
    Fetches market data for the given ticker, calculates Fibonacci retracement levels, and plots them.

    Parameters
    ----------
        ticker (str): The ticker symbol for which to fetch market data and plot Fibonacci levels.
    """
    global trend, levels

    try:
        # Assuming you have a method to get market history data
        symbol = client.get_symbol(ticker)
        data = client.get_market_history(symbol=symbol.id, resolution=Resolution.H1)
        
        # Create a DataFrame from the retrieved data
        df = pd.DataFrame([bar.model_dump() for bar in data])
        df['time'] = pd.to_datetime(df['time'], utc=True)
        df.set_index('time', inplace=True)
        
        # Display the initial candlestick chart
        fig = go.Figure()
        fig.add_trace(go.Candlestick(
            x=df.index,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Candlestick'
        ))
        fig.show()
        
        trend = None
        levels = None

        # Widgets for date range input and trend selection
        frm_input = widgets.Text(description='From:', placeholder='YYYY-MM-DD HH:MM:SS')
        to_input = widgets.Text(description='To:', placeholder='YYYY-MM-DD HH:MM:SS')
        trend_input = widgets.RadioButtons(
            options=['uptrend', 'downtrend'],
            description='Trend:',
            disabled=False
        )
        button = widgets.Button(description="Show ")

        def update_levels(button):
            global trend, levels
            try:
                frm = pd.to_datetime(frm_input.value, utc=True)
                to = pd.to_datetime(to_input.value, utc=True)
                fibonacci_df = df.loc[frm:to]
                max_value = fibonacci_df['high'].max()
                min_value = fibonacci_df['low'].min()
                trend = True if trend_input.value == 'uptrend' else False
                levels = get_fibonacci_levels(max_value, min_value, trend)
                if levels:
                    print("Percentage level\t Price")
                    for level, price in levels.items():
                        print(f'{level}\t\t {round(price, 3)}')
                    plot_fibonacci_retracement(df, fibonacci_df, levels, ticker)
            except Exception as e:
                print(f"Error: {e}")

        button.on_click(update_levels)
        display(VBox([frm_input, to_input, trend_input, button]))

    except ValueError:
        print('Record not found')
    except Exception as e:
        print(f'Error: {str(e)}')


In [9]:
# Call the function for a specific ticker symbol and date range
fib_retrace('EURUSD')  # Replace with the desired ticker symbol 


Percentage level	 Price
0.00%		 1.096
23.6%		 1.094
38.2%		 1.093
50.0%		 1.092
61.8%		 1.091
100.0%		 1.087


### Example Plot

Below is a static image of Fibonacci Retracement applied to EUR/USD historical data:
![Fibonacci Plot](img/Fibonacci_plot.png)